## Simulator (fleet of uavs)

This notebook serves to get familiar with the simulator

In [ ]:
from config import Color
from helpers import clean
from helpers.connections.mavlink.customtypes.location import ENUPose, GRAPose
from plan import Plan
from simulator import (
    QGC,
    ConfigGazebo,
    ConfigNovis,
    ConfigQGC,
    Gazebo,
    NoVisualizer,
    Simulator,
)

clean()

## Simulation Configuration

We can easily extend the configuration from Notebook 2 to multiple UAVs. 
We are also adding here a mission delay in secons.

In [12]:
gra_origin = GRAPose(lat=-35.3633280, lon=149.1652241,alt=0,heading=90) 
enu_origin = ENUPose(x=0, y=0, z=gra_origin.alt, heading=gra_origin.heading) 

base_homes= ENUPose.list([  # east, north, up, heading
    (0., 15., 0., 0.),
    (15., 0., 0., 0),
    (5., -20., 0., 30.),
    (-15., -15., 0., 0),
    (-15., 0., 0., 45),

])
base_paths = [Plan.create_square_path(side_len=10, alt=5,heading=0) for _ in base_homes]

colors=[
    Color.BLUE,
    Color.GREEN,
    Color.BLACK,
    Color.ORANGE,
    Color.RED,
]
msn_delays=[0,1,2,3,4] # in seconds

## Visualizer

Extending the visualizer configuration to multiple UAVs is also straightforward.

## Gazebo

In [13]:
gaz_config = ConfigGazebo(origin = enu_origin,
                          world_path="simulator/gazebo/worlds/runway.world")

for path,home,c in zip(base_paths,base_homes,colors):
    gaz_config.add(base_path=path,base_home=home,color=c)
gaz_config.show()

gaz= Gazebo(gaz_config,gra_origin)

## QGroundControl

In [14]:
qgc_config = ConfigQGC(origin = gra_origin)

for path,home,color,delay in zip(base_paths,base_homes,colors,msn_delays):
    qgc_config.add(base_path=path,base_home=home,color=color,mission_delay=delay)
qgc_config.show()

qgc=QGC(qgc_config)

## No Visualizer

In [15]:
novis_config = ConfigNovis(origin=gra_origin)
for home in base_homes:
    novis_config.add(base_home=home)
    
novis=NoVisualizer(novis_config)

## Launch Simulation

**Here we configure the simulation:**
* **visualizers**: Select the desired visualizer for the simulation (only one visualizer at a time is currently supported).    
* **missions**: List of missions for each UAV. Although advanced, customized missions can be built using the `Mission` class defined in the QGroundControl configuration script, here we use the QGroundControl configuration builder for simplicity.  
* **terminals**: Processes to display in the terminal. The available processes are defined in the `sim.py` script: `["launcher", "veh", "logic", "proxy", "gcs"]`.  
* **suppress_output**: Processes whose output should be suppressed. The available processes are defined in the `sim.py` script: `["launcher", "veh", "logic", "proxy", "gcs"]`.  
* **verbose**: Verbosity level (0 = no information, 1 = normal information, 2 = include debugging information).  
* **other arguments**: More advanced options that allow further customization.  


In [16]:
qgc_config.vehicles

[QGCVehicle(home=GRAPose(lat=-35.36332799988755, lon=149.16505886598486, alt=0.0, heading=90.0), mission=Mission(traj=[QGCWP(pos=GRA(lat=-35.36332799988755, lon=149.16505886598486, alt=5.0), color=blue), QGCWP(pos=GRA(lat=-35.363327999687634, lon=149.1649487099748, alt=5.0), color=blue), QGCWP(pos=GRA(lat=-35.36323816763811, lon=149.16494871028124, alt=5.0), color=blue), QGCWP(pos=GRA(lat=-35.36323816783802, lon=149.1650588661687, alt=5.0), color=blue), QGCWP(pos=GRA(lat=-35.36332799988755, lon=149.16505886598486, alt=5.0), color=blue)], delay=0, n_items=7)),
 QGCVehicle(home=GRAPose(lat=-35.36319325192571, lon=149.1652241, alt=0.0, heading=90), mission=Mission(traj=[QGCWP(pos=GRA(lat=-35.36319325192571, lon=149.1652241, alt=5.0), color=green), QGCWP(pos=GRA(lat=-35.36319325187572, lon=149.16511394417375, alt=5.0), color=green), QGCWP(pos=GRA(lat=-35.363103419826196, lon=149.16511394429637, alt=5.0), color=green), QGCWP(pos=GRA(lat=-35.36310341987617, lon=149.1652241, alt=5.0), color=g

In [17]:
simulator = Simulator(
	visualizers=[qgc],
	gcs_system_ids={'multicolor':[1,2,3,4,5]},
	missions=[veh.mission for veh in qgc_config.vehicles],
	terminals=['gcs'],
	verbose=1,
)
orac = simulator.launch()

15:53:43 - Oracle ⚪ - INFO - 🚀 GCS multicolor launched (PID 1669011)
15:53:44 - Oracle ⚪ - INFO - 🗺️  QGroundControl launched for 2D visualization — simulation powered by ArduPilot SITL.


## Oracle checking

In [18]:
orac.run()

15:53:44 - Oracle ⚪ - INFO - 🏁 Starting Oracle with 5 vehicles and 1 GCSs
15:55:11 - Oracle ⚪ - INFO - Received message 'DONE' from GCS multicolor
15:55:11 - Oracle ⚪ - INFO - GCS multicolor removED. Remaining GCS: 0
15:55:11 - Oracle ⚪ - INFO - ✅ Main monitoring loop completed - all connections closed
15:55:11 - Oracle ⚪ - INFO - 🎉 Oracle shutdown complete!
